In [1]:
import pandas
import numpy as np
import os
from pathlib import Path

from tqdm.notebook import tqdm

In [8]:
import glob
from shutil import copyfile

img_path = r'/home/pub/Documents/mnt/Shreyansh/20200804/B16OVA_OT1/'
dest_path = r'/home/pub/Documents/mnt/Gustave/MIOCS_code/image_analysis/example_images'

time_range = np.arange(400,410,1)
color_range = [1,2]
m = 20

for t in time_range:
    for c in color_range:

        fname = f'c{c:01d}t{t:03d}m{m:02d}.tif'
        copyfile(os.path.join(img_path, fname), 
                 os.path.join(dest_path, fname))



In [8]:
img_path = r'/Volumes/Multicell/Shreyansh/20200312/TIFF'
save_path = r'/Volumes/Multicell/Shreyansh/20200312'

def make_img_folder(img_path, save_path):
    
    new_folder = os.path.join(save_path, 'imges')
    
    Path(new_folder).mkdir(parents=True, exist_ok=True)
    
    for file in tqdm(sorted(os.listdir(img_path))):

        file_name, _ = file.split('.')

        _, m = file_name.split('xy')
        m, c = m.split('c')
        c, t = c.split('t')
                
        m = int(m)
        t = int(t)
        c = int(c)

        fname = f'm{m:02d}t{t:02d}c{c}.tif'
        
        if fname not in os.listdir(new_folder):

            os.rename(os.path.join(img_path, file),
                        os.path.join(new_folder, fname))
            
    return

In [ ]:
make_img_folder(img_path,save_path)

## Preparing the local environment

In [ ]:
import sys

sys.path.insert(0,'./image-analysis/api')

In [ ]:
from api import read
from api import segment
from api import analyse

In [ ]:
import api.segment as segment

## Get the virtual stack

In [ ]:
img_path = r'/Users/gustaveronteix/Documents/Projets/Projets Code/imges_test_image_analysis/imges'

In [ ]:
vs = read.VirtualStack(img_path)

In [ ]:
vs.ranges['t']['max']

In [ ]:
c1_time_reader = vs.read(t = None, m = None, c = 1)
c2_time_reader = vs.read(t = None, m = None, c = 2)

In [ ]:
import matplotlib.pyplot as plt

from api.segment import segment
from api.analyse import analyse

In [ ]:
img_fluo = vs.get_single_image(m=10, t=315, c=2)
img_BF = vs.get_single_image(m=10, t=315, c=1)

In [ ]:
crop_img_BF = segment.select_well(img_BF.array, img_BF.array, 430, 430, 3)
crop_img_Fluo = segment.select_well(img_BF.array, img_fluo.array, 430, 430, 3)

In [ ]:
sph_img = segment.find_spheroid(crop_img_BF, 430, 3)

In [ ]:
df = pandas.DataFrame()

for img in c1_time_reader:
        
    t = img.meta['t']
    m = img.meta['m']

    img_fluo = vs.get_single_image(m=m, t=t, c=2)
    
    crop_img_BF = segment.select_well(img.array, img.array, 430, 430, 3)
    crop_img_Fluo = segment.select_well(img.array, img.array, 430, 430, 3)
        
    sph_img = segment.find_spheroid(crop_img_BF, 430, 3)
    
    
    timeFrame = analyse.spheroid_properties(sph_img)
    timeFrame['t'] = t
    timeFrame['m'] = m
    
    df = df.append(timeFrame)